# Forces Acting Within a Synestia

## Balance of Forces

In [ ]:
#diagram here

To understand the dynamics of a synestia, we must first understand its mechanics (balance of forces). There are three forces at work: the centripetal force (arises from circular motion), the pressure force from the gas, and gravity of components of the structure. A parcel of gas in a synestia is stably orbiting when it follows the following equation:
$$\omega^2r = \frac{1}{\rho}\frac{dP}{dr} + \frac{GM}{r^2}(1 + \frac{3J_2a_{eq}^2}{2r^2}(\frac{3z^2}{r^2} - 1))$$
where $\omega$ is the angular velocity of the parcel, r=(x,y,z) is the distance from the parcel to the center of the synestia (origin), $\rho$ is the density of the parcel of gas, dP/dr is the radial pressure gradient at the location of the parcel, G is the gravitational constant, M is the mass of the synestia the parcel is orbiting in, J$_2$ is the oblateness of that synestia, a$_{eq}$ is the equatorial radius of this synestia, and z is the vertical distance off the midplane of the parcel of gas.

The angular velocity of a parcel of gas in equilibrium (gravitational force, pressure force, and circular motion are balanced so that the particle maintains a stable orbit) will change according to the strength of the gravitational field and the pressure gradient surrounding it in a synestia. The gas density, gas pressure, and second-order gravity are significant forces in shaping the oblate, flared gas structure of a synestia and motion of moonlets inside a synestia.

### Gravity

The more massive a synestia is, the larger the equatorial radius of the planet-like region of a synestia is, and the more oblate a synestia is, then the stronger the gravitational field is in that synestia. The stronger the gravitational field, the higher angular velocity the gas particle inside that synestia must have in order to maintain its stable orbit.

The local strength of the gravity field of a synestia will vary based on a gas parcel's location in a synestia. The strongest gravitational forces will be along the midplane and closest to the center of a synestia. Both are considered equilibrium points (where a particle will settle when forces are balanced). As a gas parcel's distance from the rotation axis and midplane increase, the magnitude of gravity will decrease. 

### Pressure Force

Density of the gas will also vary based on the parcel's location in a synestia. There is a radial dependency for density. Density decreases off the midplane to higher z, and also decreases at larger distances from a synestia's rotational axis, cylindrical radius r$_{xy}$. Density follows a negative power law dependence in r$_{xy}$ and an exponential decay dependence in z. Density will typically follow the form:
$$\rho = \rho(r_{xy}, z) = \rho_0r_{xy}^{-a}e^{-z^2/z_s^2}$$
where $\rho_0$ is a reference density in the midplane, r$_{xy}$ is the cylindrical radius of the gas parcel and makes the base of the power law fit for the midplane, a is the exponent of the midplane's power law fit, z is the vertical position of the parcel of gas, and z$_s$ is the scale height. The scale height is a measure of how quickly the density decreases in z. It measures the e-folding time, or at what z the density reaches 1/e th of its midplane value. The scale height has a power law dependency on cylindrical radius $r_{xy}$ with the form:
$$z_s = br_{xy}^c$$
where b is a coefficent and c is the exponent of the power law.

Essentially, gas density and pressure exponentially decay from the midplane to the photosphere (cloud layer) of a synestia, and will do so more quickly at larger distances from a synestia's rotational axis. In the midplane, gas density and pressure decrease with increasing cylindrical radius according to some power law. This shapes a synestia's pressure and density fields into nested, flared contours.

The greater the density of the gas field and the pressure differential across the gas parcel [one side of the gas particle (smaller radius) has a much stronger pressure than the other side (larger radius)], the larger the pressure support will be for an orbiting gas particle. With increased pressure support, a gas parcel can orbit stably at lower angular velocities. We call these sub-Keplerian orbits. 

## Sub-Keplerian Orbits of Moonlets in Synestias: Balanced by Oblate Gravity and Pressure

### Oblate Gravity Term

You can see how a moon travels when only a synestia's gravity (first and second order) is at play, so that we have a baseline to compare to. The oblateness of a synestia's gravity field is non-negligible. A synestia's gravity field has extra gravitational acceleration closest to both the midplane and center of a synestia due to the oblateness of a synestia. Notice how any orbits off the midplane are affected by the J$_2$ term in a synestia's gravity. The orbit of a moonlet within the gravity field of an Earth-mass synestia is shown below. The equivalent Keplerian orbit (first order gravity only) is shown for reference.

In [ ]:
#J2 gravity orbits
#put with gas drag, so can compare side-by-side
import rebound
from syndef import synfits #import synestia snapshot (impact database)

Mass_syn=np.sum(synfits.SNAP_CukStewart.m) #Earth mass synestia in kg

def pltKep(rad_moon,sma,i,ecc):
    #rad_moon is the bodily radius of the moon
    #i is the inclination of the moon's orbital plane
    #sma is the semi-major axis of the moon's orbit
    #ecc is the eccentricity of the moon's orbit
    m_moon = 4.*math.pi*dens*(rad_moon**3)/3.
    sim = rebound.Simulation() #start simulation
    sim.units = ('Hr', 'M', 'Kg') #use SI units
    sim.add(m=Mass_syn) #add central body
    sim.add(m=m_moon,a=sma,inc=i,e=ecc)
    fig,ax_main,ax_sub1,ax_sub2 = rebound.OrbitPlot(sim,slices=1,xlim=[-120000000,60000000],ylim=[-60000000,60000000],unitlabel='(m)')

J2Ma2 = synfits.SNAP_Canup.J2Ma2
def J2(reb_sim):
    ps = reb_sim.particles
    r_orb = (ps[1].x**2 + ps[1].y**2 + ps[1].z**2)**0.5
    costheta = ps[1].z/r_orb
    grav2 = 3.0*reb_sim.G*J2Ma2*(r_orb**(-5))
    ps[1].ax -= ps[1].x*grav2*( 2.5*costheta*costheta - 0.5 )
    ps[1].ay -= ps[1].y*grav2*( 2.5*costheta*costheta - 0.5 )
    ps[1].az -= ps[1].z*grav2*( 2.5*costheta*costheta - 1.5 )

from rebound.plotting import fading_line
def pltJ2(rad_moon,sma,i,ecc):
        #rad_moon is the bodily radius of the moon
    #i is the inclination of the moon's orbital plane
    #sma is the semi-major axis of the moon's orbit
    #ecc is the eccentricity of the moon's orbit
    m_moon = 4.*math.pi*dens*(rad_moon**3)/3.
    sim_kep = rebound.Simulation() #start simulation
    sim_kep.units = ('Hr', 'M', 'Kg') #use SI units
    sim_kep.add(m=Mass_syn) #add central body
    sim_kep.add(m=m_moon,a=sma,inc=i,e=ecc)
    ps_kep = sim_kep.particles
    kep_moon = ps_kep[1]
    kep_orbit = np.asarray(kep_moon.sample_orbit())
    sim_j2 = rebound.Simulation() #start simulation
    sim_j2.units = ('Hr', 'M', 'Kg') #use SI units
    sim_j2.add(m=Mass_syn) #add central body
    sim_j2.add(m=m_moon,a=sma,inc=i,e=ecc)
    sim_j2.additional_forces = J2
    ps_j2 = sim_j2.particles
    #j2_moon = ps_j2[1]
    #j2_orbit = np.asarray(j2_moon.sample_orbit())
    N = 100
    j2_x = np.zeros(N)
    j2_y = np.zeros(N)
    j2_z = np.zeros(N)
    times = np.linspace(0.,1.,N)
    for k,time in enumerate(times):
        sim_j2.integrate(time)
        j2_x[k] = ps_j2[1].x
        j2_y[k] = ps_j2[1].y
        j2_z[k] = ps_j2[1].z
    fig,ax_main,ax_sub1,ax_sub2 = rebound.OrbitPlot(sim_j2,slices=1,xlim=[-120000000,60000000],ylim=[-60000000,60000000],unitlabel='(m)',color=True)
    lc_main = fading_line(kep_orbit[:,0], kep_orbit[:,1], linewidth=1)
    lc_sub1 = fading_line(kep_orbit[:,0], kep_orbit[:,2], linewidth=1)
    lc_sub2 = fading_line(kep_orbit[:,2], kep_orbit[:,1], linewidth=1)
    ax_main.add_collection(lc_main)
    ax_sub1.add_collection(lc_sub1)
    ax_sub2.add_collection(lc_sub2)
    
style = {'description_width': 'initial'}
layout = {'width': '400px'}
interact(pltJ2,
         rad_moon=FloatSlider(value=1, min=1, max=100, step=10, description='Body radius of moonlet (m)',
                            continuous_update=False, readout=True, readout_format='.1f', style=style, layout=layout),
         sma=FloatSlider(value=20e6, min=10e6, max=60e6, step=10e6, description='Semi-major axis (m)',
                        continuous_update=False, readout=True, readout_format='.1f', style=style, layout=layout),
         i=FloatSlider(value=0, max=2.*math.pi, step=0.2, description='Inclination of orbit (radians)',
                        continuous_update=False, readout=True, readout_format='.1f', style=style, layout=layout),
         ecc=FloatSlider(value=0, max=0.9, step=0.1, description='Eccentricity of orbit (unitless)',
                        continuous_update=False, readout=True, readout_format='.1f', style=style, layout=layout),
         )

### Pressure Term

The pressure gradient term is significant in a synestia and allows a parcel of gas to orbit at a speed slower than would be allowed with gravity only (Keplerian). Parcels of gas in a synestia orbit at sub-Keplerian velocities. In other words, the angular velocity is lower than what it would be for a Keplerian orbit.

To maintain a stable (no inward or outward spiralling) elliptical orbit (forces are balanced), an orbiting body must move fast enough around its central body to resist the inward pull of the total sum of forces (typically gravity, but pressure also acts in a synestia). For a Keplerian orbit, the only force that is acting (and is balanced) is gravity. In the non-corotating region of a synestia, the material experiences a sub-Keplerian orbit. There is sufficient pressure support to counteract gravity in z and lessen the strength of gravity in the $r_{xy}$ direction. The strength of the sum of inward forces is less than it would be for a Keplerian orbit. This allows the gas in a synestia to orbit at an angular velocity that is slower than it would be for an equivalent Keplerian orbit.

Take a look below at various sub-Keplerian orbits of gas particles within a synestia. Notice that the mass of the parcel of gas does not matter and is not included as an input.

In [ ]:
from syndef import synfits
import numpy as np
from syndef import synfits
import matplotlib.pyplot as plt
rxy_Kep = np.linspace(0.,300.e6,100)
angvel_Kep = ((G*np.sum(synfits.SNAP_CukStewart.m)/rxy_Kep**3)**0.5)*3600*180/math.pi
fig,ax1 = plt.subplots(figsize=(8,4))
plt.title('Angular Velocity Profile of a Synestia')
ax1.plot(synfits.SNAP_CukStewart.rxy[synfits.SNAP_CukStewart.ind_mid]/1e3,synfits.SNAP_CukStewart.omg_z[synfits.SNAP_CukStewart.ind_mid]*3600*180/math.pi,'.')
ax1.plot(rxy_Kep/1e3,angvel_Kep,'--',label='Keplerian orbit')
ax1.set_xlabel('Distance from Rotational Axis (km)')
ax1.set_ylabel('Angular Velocity (degrees/hr)',color='tab:blue')
ax1.tick_params(axis='y', labelcolor='tab:blue')
ax1.set_ylim([np.amin(synfits.SNAP_CukStewart.omg_z[synfits.SNAP_CukStewart.ind_mid]*3600*180/math.pi),np.amax(synfits.SNAP_CukStewart.omg_z[synfits.SNAP_CukStewart.ind_mid]*3600*180/math.pi)])
plt.xlim([0.1,100.e3])
plt.legend()
ax2 = ax1.twinx()
period = 2.*math.pi/(synfits.SNAP_CukStewart.omg_z[synfits.SNAP_CukStewart.ind_mid]*3600)
ax2.plot(synfits.SNAP_CukStewart.rxy[synfits.SNAP_CukStewart.ind_mid]/1e3,period,'.',color='grey')#,alpha=0)
ax2.set_ylabel('Corresponding Orbital Period (hr)',color='dimgrey')
ax2.set_ylim([np.amin(period),250])
ax2.tick_params(axis='y', labelcolor='dimgrey')
plt.show()

In [ ]:
#draw interactive below
#angular velocity profile with length of day (rotation speed)
#have Keplerian
#another section: why make very puffy, very big
#INSERT FIGURE OF ANGULAR VELOCITY PROFILE OF SYNESTIA; CONTRAST WITH LINEAR VELOCITY

#how does pressure change? slower velocity at that location
#plot difference, slider bar for pressure effect
#put in the equation: adjustable parameter (adding pressure support)
#how does this change

In [ ]:
#plot of orbit (shows 1 orbit) with J2 and pressure forces added, how long orbital period is,
#change pressure term and see how that affects angular velocity profile

This is a plot of the midplane (z = 0) angular velocity profile. The profile is continuous because there are no dynamically distinct features of a synestia. It is a connected body. Notice that the profile is flat at smaller radii. This is the corotating portion of a synestia. All material rotates about its axis together at the same (rapid) angular velocity. At larger radii, the profile drops off, where material in a synestia can no longer corotate.

#### Hydrostatic Equilibrium

The z component of the pressure gradient allows a particle to orbit at the same z. In other words, gas parcels experience hydrostatic equilibrium in z. The parcel of gas's gravitational pull towards the midplane is balanced by the vertical pressure gradient. Gas parcels will not move in the z direction; they have zero movement in z. Hydrostatic equilibrium in z follows this equation:
$$\frac{dP}{dz} = -\rho g_z = -\rho(r_{xy}, z)\frac{3GJ_2Ma_{eq}^2z}{r^5}(\frac{2.5z^2}{r^2} - 1.5)$$
where g$_z$ is the z component of the gravitational acceleration acting at a particular location r = (x, y, z) in a synestia with equatorial radius a$_{eq}$, total mass M, and J$_2$ factor based on its mass distribution (oblateness).

The magnitude of the pressure gradient in z of the gas in a synestia will change according to the magnitude of the gas density and gravitational acceleration in z at a particular r$_{xy}$ = (x, y), z location. The pressure gradient term in the balance of forces in a synestia is responsible for the puffiness of a synestia. It causes a synestia to flare and take on a disk-like shape. 

Let's take a look below at the pressure field of an example Moon-forming synestia. If a synestia is in hydrostatic equilibrium, it should look like a flared disk and have the same contours as would be expected for hydrostatic equilibrium.

In [1]:
#plot of pressure contours
#plot of differential between pressure contours and hydrostatic equilibrium

As you can see, the pressure contours match nicely with those expected for hydrostatic equilibrium. They also line up with the shape of the synestia, as is expected since the pressure contours are a result of the gas structure in a synestia.